*Data Cleaning for Covid Clinical Studies*

In [177]:
import pandas as pd
import numpy as np

In [178]:
pd.set_option('display.max_columns', None)
df = pd.read_csv('C:/Users/GUORUI/Desktop/Graduate Bentley-desktop/2021 Spring/MA705/ClinicalStudies.csv')

**detect NA and fill NA in Phases Column **

In [179]:
df.Phases = df.Phases.fillna(value = 'No Defined Phase')
df.URL = df.URL.fillna(value = 'https://ClinicalTrials.gov/show/NCT04387656')
#df[df.Phases.isna()]
#df.isna().sum()

In [180]:
len(df)

1266

**replace values for Phases column**

In [181]:
df = df.replace({'Phases': {'Not Applicable': 'No Defined Phase','Early Phase 1':'Phase 0'}})
#df.Phases.unique()


** create new recruiting column for Recruiting status **

In [182]:
Recruiting_Status = []

for row in df['Status']:
    if row =='Recruiting':
        Recruiting_Status.append('Recruiting')
    elif (row =='Active, not recruiting' or row == 'Not yet recruiting' or row == 'Suspended'):
        Recruiting_Status.append('Potential Recruiting')
    elif row =='Available':
        Recruiting_Status.append('Other Recruiting')
    else:
        Recruiting_Status.append('Not Recruiting')

df['Recruiting_Status'] = Recruiting_Status

In [183]:
#check values
#pd.value_counts(df['Recruiting_Status'])

In [184]:
#df = df["Enrollment"].astype(int)

**extract state, city and authorities**

In [185]:
ldf = df.loc[:,['NCT Number','Locations']]

**split multiple locations and create new row for each location**

In [186]:
new_ldf = pd.DataFrame(ldf.Locations.str.split('|').tolist(), index=df['NCT Number']).stack()
new_ldf = new_ldf.reset_index([0, 'NCT Number'])
new_ldf.columns = ['NCT Number', 'Location']
new_ldf

,NCT Number,Location
0,NCT04482621,"Johns Hopkins University, Baltimore, Maryland,..."
1,NCT04425538,"Tufts Medical Center, Boston, Massachusetts, U..."
2,NCT04354831,"Froedtert Hospital, Milwaukee, Wisconsin, Unit..."
3,NCT04388683,"Tufts Medical Center, Boston, Massachusetts, U..."
4,NCT04628039,VA Puget Sound Health Care System Seattle Divi...
...,...,...
10245,NCT04666597,"Universidad Peruana Cayetano Heredia, Lima, Peru"
10246,NCT04666597,"University of Porto, Porto, Portugal"
10247,NCT04666597,"University of Moscow, Moscow, Russian Federation"
10248,NCT04666597,"University of Sheffield, Sheffield, United Kin..."


**merge two dataframe   **

In [187]:
#combine new location dataframe with the entire df
df = pd.merge(new_ldf,df,how='left',on = 'NCT Number')
df

,NCT Number,Location,Title,Status,Gender,Phases,Enrollment,Locations,URL,Recruiting_Status
0,NCT04482621,"Johns Hopkins University, Baltimore, Maryland,...",Decitabine for Coronavirus (COVID-19) Pneumoni...,Recruiting,All,Phase 2,40.0,"Johns Hopkins University, Baltimore, Maryland,...",https://ClinicalTrials.gov/show/NCT04482621,Recruiting
1,NCT04425538,"Tufts Medical Center, Boston, Massachusetts, U...",A Phase 2 Trial of Infliximab in Coronavirus D...,Completed,All,Phase 2,17.0,"Tufts Medical Center, Boston, Massachusetts, U...",https://ClinicalTrials.gov/show/NCT04425538,Not Recruiting
2,NCT04354831,"Froedtert Hospital, Milwaukee, Wisconsin, Unit...",A Study Evaluating the Efficacy and Safety of ...,"Active, not recruiting",All,Phase 2,131.0,"Froedtert Hospital, Milwaukee, Wisconsin, Unit...",https://ClinicalTrials.gov/show/NCT04354831,Potential Recruiting
3,NCT04388683,"Tufts Medical Center, Boston, Massachusetts, U...",Inhaled Nitric Oxide for Preventing Progressio...,"Active, not recruiting",All,Phase 2,42.0,"Tufts Medical Center, Boston, Massachusetts, U...",https://ClinicalTrials.gov/show/NCT04388683,Potential Recruiting
4,NCT04628039,VA Puget Sound Health Care System Seattle Divi...,Chronic Lung Disease and COVID-19: Understandi...,Not yet recruiting,All,No Defined Phase,506.0,VA Puget Sound Health Care System Seattle Divi...,https://ClinicalTrials.gov/show/NCT04628039,Potential Recruiting
...,...,...,...,...,...,...,...,...,...,...
10245,NCT04666597,"Universidad Peruana Cayetano Heredia, Lima, Peru",Pandemic-adapted Caries Care Multicentre Case ...,Recruiting,All,No Defined Phase,420.0,"Indiana University, Indianapolis, Indiana, Uni...",https://ClinicalTrials.gov/show/NCT04666597,Recruiting
10246,NCT04666597,"University of Porto, Porto, Portugal",Pandemic-adapted Caries Care Multicentre Case ...,Recruiting,All,No Defined Phase,420.0,"Indiana University, Indianapolis, Indiana, Uni...",https://ClinicalTrials.gov/show/NCT04666597,Recruiting
10247,NCT04666597,"University of Moscow, Moscow, Russian Federation",Pandemic-adapted Caries Care Multicentre Case ...,Recruiting,All,No Defined Phase,420.0,"Indiana University, Indianapolis, Indiana, Uni...",https://ClinicalTrials.gov/show/NCT04666597,Recruiting
10248,NCT04666597,"University of Sheffield, Sheffield, United Kin...",Pandemic-adapted Caries Care Multicentre Case ...,Recruiting,All,No Defined Phase,420.0,"Indiana University, Indianapolis, Indiana, Uni...",https://ClinicalTrials.gov/show/NCT04666597,Recruiting


In [188]:
#new[new['NCT Number']== 'NCT04326309']

In [189]:
df.isna().sum()

NCT Number             0
Location               0
Title                  0
Status                 0
Gender                 0
Phases                 0
Enrollment           411
Locations              0
URL                    0
Recruiting_Status      0
dtype: int64

In [190]:
df[df['NCT Number']== 'NCT04326309'].Location.str.rsplit(',').str[-1]

9830     United States
Name: Location, dtype: object

**split location, find countries**

In [191]:
df['Country'] = df['Location'].str.rsplit(', ').str[-1]

In [192]:
df['Country'].unique()

array(['United States', 'United Kingdom', 'Canada', 'Spain', 'India',
       'Sweden', 'Uganda', 'Mexico', 'Egypt', 'Islamic Republic of',
       'Brazil', 'Puerto Rico', 'Germany', 'Hungary', 'Belgium', 'France',
       'Argentina', 'Chile', 'South Africa', 'Ukraine', 'Romania',
       'Netherlands', 'Switzerland', 'Denmark', 'Republic of', 'Japan',
       'Singapore', 'Greece', 'Peru', 'Colombia', 'Russian Federation',
       'Italy', 'China', 'Hong Kong', 'Taiwan', 'Portugal', 'Bulgaria',
       'Ireland', 'Poland', 'Kenya', 'Turkey', 'Austria', 'Israel',
       'Finland', 'Philippines', 'Honduras', 'Australia', 'Malawi',
       'Zambia', 'Zimbabwe', 'Saint Lou', 'Botswana', 'Mozambique',
       'Tanzania', 'Afghanistan', 'Djibouti', 'Guam', 'Iraq', 'Kuwait',
       'Cyprus', 'Czechia', 'Estonia', 'Iceland', 'Lithuania', 'Slovakia',
       'Slovenia', 'Serbia', 'Bolivia', 'New Zealand', 'Nigeria', 'Ghana',
       'Croatia', 'Ethiopia', 'Armenia', 'Costa Rica', 'Guatemala',
       'L

**only keep studies in United States**

In [193]:
df = df.loc[df['Country']=='United States']
#only contain united states
len(df)

7625

**split location, find states,cities,authorities**

In [194]:
df['State'] = df['Location'].str.rsplit(', ').str[-2]
df['City'] = df['Location'].str.rsplit(', ').str[-3]
df['Authority'] = df['Location'].str.split(', ').str[0]

<ipython-input-194-847f2eae1c83>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['State'] = df['Location'].str.rsplit(', ').str[-2]
<ipython-input-194-847f2eae1c83>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['City'] = df['Location'].str.rsplit(', ').str[-3]
<ipython-input-194-847f2eae1c83>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyd

**replace values for Phases column**

In [195]:
df = df.replace({'Phases': {'Phase 1|Phase 2': 'Phase 1 and Phase 2','Phase 2|Phase 3':'Phase 2 and Phase 3'}})
#df.Phases.unique()

**replace values for Gender column**

In [196]:
df = df.replace({'Gender': {'Female': 'Female Only','Male':'Male Only'}})
#df.Gender.unique()

In [197]:
df.isna().sum()

NCT Number             0
Location               0
Title                  0
Status                 0
Gender                 0
Phases                 0
Enrollment           279
Locations              0
URL                    0
Recruiting_Status      0
Country                0
State                  0
City                   0
Authority              0
dtype: int64

In [198]:
len(df)

7625

In [199]:
df = df.drop(['Location', 'Locations'], axis=1)

In [200]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7625 entries, 0 to 10230
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   NCT Number         7625 non-null   object 
 1   Title              7625 non-null   object 
 2   Status             7625 non-null   object 
 3   Gender             7625 non-null   object 
 4   Phases             7625 non-null   object 
 5   Enrollment         7346 non-null   float64
 6   URL                7625 non-null   object 
 7   Recruiting_Status  7625 non-null   object 
 8   Country            7625 non-null   object 
 9   State              7625 non-null   object 
 10  City               7625 non-null   object 
 11  Authority          7625 non-null   object 
dtypes: float64(1), object(11)
memory usage: 774.4+ KB


In [201]:
df.tail()

,NCT Number,Title,Status,Gender,Phases,Enrollment,URL,Recruiting_Status,Country,State,City,Authority
10192,NCT03474965,Study of Dose Confirmation and Safety of Criza...,Recruiting,All,Phase 2,100.0,https://ClinicalTrials.gov/show/NCT03474965,Recruiting,United States,South Carolina,Charleston,Medical University of South Carolina Medical U...
10193,NCT03474965,Study of Dose Confirmation and Safety of Criza...,Recruiting,All,Phase 2,100.0,https://ClinicalTrials.gov/show/NCT03474965,Recruiting,United States,Tennessee,Memphis,St. Jude Children's Research Hosptial
10194,NCT03474965,Study of Dose Confirmation and Safety of Criza...,Recruiting,All,Phase 2,100.0,https://ClinicalTrials.gov/show/NCT03474965,Recruiting,United States,Texas,Fort Worth,Cook Childrens Medical Center Oncology
10229,NCT04666597,Pandemic-adapted Caries Care Multicentre Case ...,Recruiting,All,No Defined Phase,420.0,https://ClinicalTrials.gov/show/NCT04666597,Recruiting,United States,Indiana,Indianapolis,Indiana University
10230,NCT04666597,Pandemic-adapted Caries Care Multicentre Case ...,Recruiting,All,No Defined Phase,420.0,https://ClinicalTrials.gov/show/NCT04666597,Recruiting,United States,Massachusetts,Boston,Tufts University


In [202]:
df.to_pickle('covid_studies.pkl')